In [19]:
from collections import namedtuple
import os, time
from stable_baselines3.common.env_checker import check_env
from time import sleep
from collections import namedtuple
import os, time
import gymnasium as gym
from gymnasium.wrappers import NormalizeObservation
import Custom_Envs
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3 import PPO, DDPG, TD3
import numpy as np


In [14]:
a = 194.84 + 152.94+179.99+179.98 +151.54 * 3 +327.74
a

1490.11

In [12]:
a -b

143.07999999999993

In [13]:
b= 194.84 + 152.94+ 179.99+169.99+151.54+169.99+327.74
b

1347.03

### TODO

In [29]:
env_name = 'Custom_Envs/CommuteEnv-v4'
one_dim = True
best_toll_initialization = False
reward_shifting = True
train_episode =  3
simulation_day_num = 3 # the simulation days in one iteration 
train_time_steps = int(simulation_day_num * train_episode)# total train times
save_episode_train = 2 # the episode number to save train results
learning_rate = 0.001
evaluation_time_episode = 2
save_episode_test = evaluation_time_episode # the episode number to save train results

In [30]:
# Dont need change parameters
start_time = time.time()
save_dir = "./results/PPO_"+time.asctime(time.localtime(start_time))
print(" save_dir ", save_dir)
env_name = 'Custom_Envs/CommuteEnv-v4'
isExist = os.path.exists(save_dir)
if not isExist:
    os.makedirs(save_dir)

 save_dir  ./results/PPO_Wed Sep 20 18:21:47 2023


### Check Env

In [34]:
# environment initialization
env = gym.make(env_name, simulation_day_num = simulation_day_num, 
                save_episode_freq = save_episode_test, 
                train=False, save_dir = save_dir, space_shape=(4, int(12*60/5)),
                 One_dim = one_dim, 
                 Best_toll_initialization = best_toll_initialization, 
                 Reward_shifting = reward_shifting)
env = NormalizeObservation(env)

In [32]:
check_env(env)

/home/xiawu/py38/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:175: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed=58287
  logger.warn(
/home/xiawu/py38/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


KeyboardInterrupt: 

### Random actions

In [35]:
for _ in range(evaluation_time_episode):
  done  = False
  obs = env.reset()[0] 
  while not done:  # every step has same seed
    action = [env.action_space.sample()]
    print("action ", action)
    obs, reward, terminated, done, info = env.step(action[0])

/home/xiawu/py38/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:175: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed=58287
  logger.warn(


action  [array([0.28852123], dtype=float32)]
action  [array([-0.59269464], dtype=float32)]
action  [array([0.50181675], dtype=float32)]
 self.episode  0
action  [array([0.61323065], dtype=float32)]
action  [array([0.80752575], dtype=float32)]
action  [array([0.7324553], dtype=float32)]
 self.episode  1


In [25]:
np.load("/home/xiawu/RL4PT/results/PPO_Wed Sep 20 18:08:43 2023/test_result/3_ppo_action.npy")
np.load("/home/xiawu/RL4PT/results/PPO_Wed Sep 20 18:08:43 2023/test_result/3_ppo_toll.npy")


array([[[473.9449194 ,  69.26051304,   4.06592848],
        [528.57625633,  70.        ,   3.84625406],
        [540.        ,  65.23814231,   4.98941154],
        [528.18124652,  64.55583066,   5.        ],
        [460.40459991,  54.67063218,   5.        ]],

       [[540.        ,  59.83631993,   1.29984466],
        [540.        ,  68.49595631,   2.26605854],
        [506.78163171,  70.        ,   1.55509207],
        [486.17304325,  70.        ,   1.        ],
        [529.76484776,  70.        ,   1.        ]],

       [[466.84106017,  70.        ,   2.00088892],
        [350.69348479,  70.        ,   1.        ],
        [300.        ,  70.        ,   1.07695407],
        [406.66452885,  70.        ,   1.85820562],
        [434.55042243,  64.75175202,   1.        ]]])

### Train

In [11]:
print(" Start training with total steps: ", train_time_steps,)
env = gym.make(env_name, simulation_day_num = simulation_day_num, 
                save_episode_freq = save_episode_train, 
                train=True, save_dir = save_dir, space_shape=(4, int(12*60/5)),
                One_dim = False, 
                Best_toll_initialization = False, 
                Reward_shifting = True
                )
env = NormalizeObservation(env)
checkpoint_callback = CheckpointCallback(save_freq= int(20*simulation_day_num), save_path=(save_dir+"/logs/"), name_prefix="PPO")
model = PPO("MlpPolicy", env, learning_rate=learning_rate, n_steps=4, verbose=1, batch_size=3, 
            target_kl=0.05, n_epochs=2, gae_lambda=0.97,  ent_coef = 0.5, clip_range=0.2, gamma=0.99, 
            tensorboard_log=(save_dir+"/tensorboards/PPO/"))
model.learn(total_timesteps = train_time_steps, tb_log_name= "first_run", callback = checkpoint_callback)
model.save(save_dir+"/logs/"+str(train_time_steps)+"_steps")
print(model.learn)

 Start training with total steps:  120
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/xiawu/py38/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:148: UserWarning: You have specified a mini-batch size of 3, but because the `RolloutBuffer` is of size `n_steps * n_envs = 4`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 1
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=4 and n_envs=1)
  warnings.warn(


Logging to ./results/PPO_Wed Sep 20 17:45:22 2023/tensorboards/PPO/first_run_1


/home/xiawu/py38/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:175: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed=19147
  logger.warn(
/home/xiawu/RL4PT/CustomEnvs/Custom_Envs/envs/test_env.py:1115: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  tollparameters = np.array([120(2*random.random()-1)+420, 10*(2*random.random()-1)+60, 2*(2*random.random()-1)+3])
/home/xiawu/RL4PT/CustomEnvs/Custom_Envs/envs/test_env.py:1115: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  tollparameters = np.array([120(2*random.random()-1)+420, 10*(2*random.random()-1)+60, 2*(2*random.random()-1)+3])
/home/xiawu/RL4PT/CustomEnvs/Custom_Envs/envs/test_env.py:1115: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  tollparameters = np.array([120(2*random.random()-1)+420, 10*(2*random.random()-

KeyboardInterrupt: 

In [19]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=576, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=576, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=3, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

#### Testing

In [18]:
env = gym.make(env_name, simulation_day_num = simulation_day_num, 
                save_episode_freq = save_episode_test, train=False, save_dir = save_dir, 
                space_shape=(4, int(12*60/5)),
                One_dim = False, 
                Best_toll_initialization = False, 
                Reward_shifting = True
                )
env = NormalizeObservation(env)        
for i in range(evaluation_time_episode):
    done  = False
    obs = env.reset()[0] 
    while not done:  # every step has same seed
        action = model.predict(obs, deterministic=True)
        obs, reward, terminated, done, info = env.step(action[0])
print("finish test¡ng!!!!!!!!!!!!!!!!!")

/home/xiawu/py38/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:175: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed=50237
  logger.warn(


KeyboardInterrupt: 